[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/uxue-sudupe/API-adibideak/blob/main/code_examples/es/API_Opendata_Euskadi_es.ipynb)

 **APIs de Open Data Euskadi**

Open Data Euskadi ha puesto a disposición de los usuarios [algunas API](https://opendata.euskadi.eus/apis/-/apis-open-data/) (interfaz de programación de aplicaciones) con el fin de facilitar la reutilización de los datos públicos más demandados.

La descripción del funcionamiento de las API utiliza el estándar OpenAPI, lo que facilita enormemente la obtención y reutilización de datos.

OpenAPI nos indica a qué URL debemos dirigirnos para obtener los datos de interés, según una serie de parámetros que pueden ser obligatorios u opcionales. La función que utilizaremos para obtener los datos puede ser GET o POST (de la librería httr).

A continuación se presentan algunos ejemplos de consultas a 2 de las APIs.

In [ ]:
# instalamos las librerías necesarias

install.packages(c("httr", "jsonlite", "dplyr", "tidyr"))

library(httr)
library(jsonlite)
library(dplyr)
library(tidyr)

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



# 1. API de Udalmap

https://opendata.euskadi.eus/api-udalmap/?api=udalmap

La API de Udalmap proporciona acceso a más de 200 indicadores de información municipal, cuya finalidad es mostrar con detalle la realidad en los municipios de la Comunidad Autónoma de Euskadi sobre áreas como economía y competitividad, cohesión social, calidad de vida, medio ambiente y movilidad.

 **1.1. Listado de todos los indicadores de Udalmap**

Se puede obtener el catálogo de todos los indicadores de Udalmap para identificar y filtrar el indicador que nos interesa llamando a la URL (endpoint) según este patrón :

`https://api.euskadi.eus/udalmap/indicators/[lang]`

Los idiomas (lang) posibles son euskera (lang=BASQUE) y castellano (lang=SPANISH).

In [ ]:

respuesta <- GET("https://api.euskadi.eus/udalmap/indicators?lang=SPANISH")
texto<- content(respuesta, as = "text", encoding = "UTF-8")

indicadores <- fromJSON(texto, flatten = TRUE)
head(indicadores)

,id,name,_links.self.name,_links.self.href,_links.subgroup.name,_links.subgroup.href,_links.group.name,_links.group.href
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,3,Población de 16 y más años ocupada en el sector industria (%),Población de 16 y más años ocupada en el sector industria (%),https://api.euskadi.eus/udalmap/indicators/3,Estructura Económica,https://api.euskadi.eus/udalmap/subgroups/E.1,Economía / Competitividad,https://api.euskadi.eus/udalmap/groups/E
2,14,Población parada de larga duración registrada en Lanbide (% población de 16 a 64 años),Población parada de larga duración registrada en Lanbide (% población de 16 a 64 años),https://api.euskadi.eus/udalmap/indicators/14,Mercado de Trabajo,https://api.euskadi.eus/udalmap/subgroups/E.2,Economía / Competitividad,https://api.euskadi.eus/udalmap/groups/E
3,15,Índice de rotación contractual (contratos/personas),Índice de rotación contractual (contratos/personas),https://api.euskadi.eus/udalmap/indicators/15,Mercado de Trabajo,https://api.euskadi.eus/udalmap/subgroups/E.2,Economía / Competitividad,https://api.euskadi.eus/udalmap/groups/E
4,20,Porcentaje de establecimientos del sector construcción sobre el total (%),Porcentaje de establecimientos del sector construcción sobre el total (%),https://api.euskadi.eus/udalmap/indicators/20,Tejido empresarial,https://api.euskadi.eus/udalmap/subgroups/E.3,Economía / Competitividad,https://api.euskadi.eus/udalmap/groups/E
5,30,Densidad comercial minorista (&#x2030; habitantes),Densidad comercial minorista (&#x2030; habitantes),https://api.euskadi.eus/udalmap/indicators/30,Vitalidad turística y comercial,https://api.euskadi.eus/udalmap/subgroups/E.6,Economía / Competitividad,https://api.euskadi.eus/udalmap/groups/E
6,38,Deuda viva del Ayuntamiento por habitante (&#x20AC;),Deuda viva del Ayuntamiento por habitante (&#x20AC;),https://api.euskadi.eus/udalmap/indicators/38,Gestión económico financiera municipal,https://api.euskadi.eus/udalmap/subgroups/E.9,Economía / Competitividad,https://api.euskadi.eus/udalmap/groups/E


 **1.2. Obtener datos de un indicador**

Cuando sepamos el ID del indicador de interés, pediremos datos a la dirección URL según este patrón:

`https://api.euskadi.eus/udalmap/indicators/[id]?[lang]`

El código identificativo lo encontramos en la lista anterior. Tomamos como ejemplo "Superficie forestal pública" (Id = 155).

Para convertir los datos que obtendremos en formato json a una tabla, debemos realizar una serie de transformaciones.


In [ ]:
ind155 <- GET("https://api.euskadi.eus/udalmap/indicators/155?lang=SPANISH")

dat_ind155 <- fromJSON(rawToChar(ind155$content))

forestal_nested <- dat_ind155$entities %>%  full_join(dat_ind155$regions, by = c("id", "years", "name")) %>% full_join(dat_ind155$municipalities, by = c("id", "years", "name"))
forestal <- forestal_nested %>% unnest(cols = years)

mantener_columnas <- grep("^_link", names(forestal), invert = TRUE)
forestal_limpio <- forestal[, mantener_columnas]
forestal_limpio

id,name,2005,2016,2018,2019,2020,2021,2022,2023
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
01000,Araba/Álava,76.26,76.47,76.49,76.52,76.51,76.53,76.52,76.51
00000,CAE,41.05,41.67,42.11,42.06,41.98,41.95,42.13,42.07
48000,Bizkaia,23.03,23.70,23.88,23.79,23.71,23.64,24.22,24.08
20000,Gipuzkoa,19.93,21.18,21.52,21.52,21.52,21.50,21.49,21.46
48100,Arratia Nerbioi / Arratia-Nervión,20.05,21.14,20.54,20.55,20.49,20.43,20.57,20.59
20300,Debagoiena / Alto Deba,9.41,11.92,12.23,12.19,12.20,12.18,12.18,12.11
20500,Goierri,11.19,23.07,12.37,12.41,12.78,12.76,12.75,12.73
48700,Plentzia-Mungia,7.67,8.67,8.77,8.70,8.76,8.74,8.61,8.60
20600,Tolosaldea / Tolosa,29.99,34.05,30.98,30.99,30.97,30.95,30.94,30.94


# 2. API de calidad del aire



 https://opendata.euskadi.eus/api-air-quality/?api=air-quality

La API de calidad del aire permite obtener los datos registrados por la Red de Control de Calidad del Aire del Gobierno Vasco

 **2.1. Listado de todas las estaciones de la red**

Para descargar las estaciones que forman parte de la Red de Control de la Calidad del Aire haríamos lo siguiente:

Identificar el endpoint que nos interesa ("get AirQuality stations"):

`https://api.euskadi.eus/air-quality/stations?[lang]`



In [ ]:

respuesta <- GET("https://api.euskadi.eus/air-quality/stations?lang=SPANISH")
texto<- content(respuesta, as = "text", encoding = "UTF-8")

estaciones_todo<- fromJSON(texto, flatten = TRUE)
estaciones <- estaciones_todo$features
head(estaciones)

,type,geometry.type,geometry.coordinates,properties.id,properties.name,properties.address,properties.description,properties.location.municipality,properties.location.county,properties.location.municipalityId,properties.location.countyId,properties._links.self.id,properties._links.self.name,properties._links.self.href
,<chr>,<chr>,<list>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Feature,Point,"-2.619475, 42.518248",83,ELCIEGO,"C/ Gabriel Celaya, 8",NA,Elciego,Araba/Álava,022,01,83,ELCIEGO,https://api.euskadi.eus/air-quality/stations/83
2,Feature,Point,"-2.963385, 43.144029",55,LLODIO,"C/ Lamuza, s/n",NA,Laudio/Llodio,Araba/Álava,036,01,55,LLODIO,https://api.euskadi.eus/air-quality/stations/55
3,Feature,Point,"-2.393703, 42.849013",76,AGURAIN,"C/ Fueros, 11. Subcomisaría de la Ertzaintza.",NA,Agurain/Salvatierra,Araba/Álava,051,01,76,AGURAIN,https://api.euskadi.eus/air-quality/stations/76
4,Feature,Point,"-3.231733, 42.875167",79,VALDEREJO,Ctro. de Interpretación. Parque Natural de Valderejo,NA,Valdegovía/Gaubea,Araba/Álava,055,01,79,VALDEREJO,https://api.euskadi.eus/air-quality/stations/79
5,Feature,Point,"-2.672563, 42.840100",85,FARMACIA,"Paseo Universidad, 7. Fac. Farmacia UPV",NA,Vitoria-Gasteiz,Araba/Álava,059,01,85,FARMACIA,https://api.euskadi.eus/air-quality/stations/85
6,Feature,Point,"-2.667785, 42.856048",75,3 DE MARZO,"Plaza Tres de Marzo, s/n",NA,Vitoria-Gasteiz,Araba/Álava,059,01,75,3 DE MARZO,https://api.euskadi.eus/air-quality/stations/75


 **2.2. Obtener las mediciones de una estación**

Si queremos los datos de las mediciones de una determinada estación o de un territorio, necesitamos el id de la estación o del territorio, y concretar los días de inicio y de fin.

Si escogemos el endpoint para obtener las mediciones de un territorio (Get daily measurements of a county), tenemos que concretar los parametros que se encuentran entre corchetes:

`https://api.euskadi.eus/air-quality/measurements/daily/counties/{county-id}/from/{date.gt}/to/{date.lt}`


Para convertir los datos que obtendremos en formato json a una tabla, debemos realizar una serie de transformaciones.

In [ ]:
# Si tomamos las estaciones de Gipuzkoa del 1 al 31 de diciembre de 2023

# Datos
respuesta <- GET("https://api.euskadi.eus/air-quality/measurements/daily/counties/20/from/2023-12-01/to/2023-12-31?lang=SPANISH")
txt <- content(respuesta, as = "text", encoding = "UTF-8")
x <- fromJSON(txt, simplifyVector = FALSE)

# ---- Construcción ----
rows_dia <- lapply(x, function(day) {
  d <- day$date
  if (is.null(day$station) || length(day$station) == 0) return(NULL)

  rows_estacion <- lapply(day$station, function(st) {
    sid   <- st$id
    sname <- st$name

    if (is.null(st$measurements) || length(st$measurements) == 0) return(NULL)

    rows_medicion <- lapply(st$measurements, function(m) {
      data.frame(
        dia        = d,
        id_estacion  = sid,
        nombre_estacion = sname,
        contaminante  = m$name,
        value        = as.numeric(m$value),
        unidad        = m$unit,
        airquality   = if (is.null(m$airquality)) NA_character_ else m$airquality,
        stringsAsFactors = FALSE
      )
    })

    do.call(rbind, rows_medicion)
  })

  rows_estacion <- Filter(Negate(is.null), rows_estacion)
  if (length(rows_estacion) == 0) return(NULL)
  do.call(rbind, rows_estacion)
})

rows_dia <- Filter(Negate(is.null), rows_dia)

mediciones <-
  if (length(rows_dia) > 0) do.call(rbind, rows_dia) else
    data.frame(dia=character(), id_estacion=character(), nombre_estacion=character(),
               contaminante=character(), value=numeric(), unidad=character(),
               airquality=character(), stringsAsFactors = FALSE)

head(mediciones)

,dia,id_estacion,nombre_estacion,contaminante,value,unidad,airquality
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>
1,2023-12-01,73,BEASAIN,Benceno,0.55,µg/m3,NA
2,2023-12-01,73,BEASAIN,CO,0.27,mg/m3,NA
3,2023-12-01,73,BEASAIN,CO 8h,0.27,mg/m3,NA
4,2023-12-01,73,BEASAIN,Etilbenceno,0.45,µg/m3,NA
5,2023-12-01,73,BEASAIN,NO,5.00,µg/m3,NA
6,2023-12-01,73,BEASAIN,NO2,25.00,µg/m3,Sin datos
